In [1]:
import visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
rm = visa.ResourceManager()
rm.list_resources()

('ASRL3::INSTR', 'ASRL10::INSTR', 'ASRL12::INSTR')

In [3]:
dc = rm.open_resource('ASRL12::INSTR')

In [4]:
dc.read()

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [6]:
class MDrive():
    def __init__(self, add):
        self.motor = rm.open_resource(add)
        self.position = 0
    
    def move_relative(self, steps, speed=200):
        self.set_speed(speed)
        self.motor.write("MR {}".format(steps))
        
        for i in range(2):
            self.motor.read()
            
        self.position += steps
        print("move relative")
        self.wait_until_stop()
    
    def move_absolute(self, steps, speed=200):
        self.set_speed(speed)
        self.motor.write("MR {}".format(steps-self.position))
        for i in range(2):
            self.motor.read()
        self.position = steps
        self.wait_until_stop()
        print("move abosulte")
    
    def get_p(self):
        return self.position
    
    def set_p(self, p):
        self.position = p
    
    def set_zero(self):
        self.position = 0
    
    def check_stop(self):
        
        self.motor.write('PR MV')
        time.sleep(0.001)
        res = None
        for i in range(3):
            res = self.motor.read()
        res = res.strip()
        if res.strip() == '0':
            return True
        elif res.strip() == '1':
            return False
        else:
            print(res)
            raise
    
    def wait_until_stop(self, time_out=1000):
        for i in range(time_out):
            if self.check_stop():
                return
            time.sleep(0.1)
    
    def set_speed(self, speed):
        self.motor.write("VM {}".format(speed))
        self.motor.write("VI {}".format(speed))
        for i in range(4):
            self.motor.read()
    
    def read(self):
        self.motor.read()
        

In [7]:
motor = MDrive(add='ASRL12::INSTR')

In [6]:
target_position = {6: 6250, 5:8750}#
def growth(motor, target, round, speed, growth_total_steps):
    motor.set_p(0)
    target_center = target_position[target]
    if motor.get_p() < target_center:
        starting_point = target_center - (growth_total_steps) // 2
        starting_dir = 1
    else:
        starting_point = target_center + (growth_total_steps) // 2
        starting_dir = -1
        
    motor.move_absolute(starting_point)
    growth_total_steps = growth_total_steps * starting_dir
    for i in range(round):
        print("round:{}".format(i+1))
        motor.move_relative(growth_total_steps, speed)
        growth_total_steps *= -1
        
    print("growth finished")
        
        
        
        



In [28]:
motor2.read()

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [29]:
motor2 = MDrive(add='ASRL4::INSTR')
motor1 = MDrive(add='ASRL12::INSTR')

In [23]:
do_task=True

In [24]:
if not do_task:
    raise

motor.move_relative(-25000, 30)
do_task=False

move relative
PR


RuntimeError: No active exception to reraise

In [46]:
motor1.move_relative(25000, 500)

move relative


In [32]:
if do_task:
    motor1.move_relative(40000, 500)
    motor2.move_relative(1000, 50)
    do_task=False

move relative


VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [17]:
target = 5
round = 1
speed = 20  #10step / second
growth_total_steps = 2000
growth(motor, target, round, speed, growth_total_steps)

move abosulte
round:1
move relative


growth finished
